In [37]:
from tqdm import tqdm_notebook
import io
import gzip
import csv
import networkx as nx

In [3]:
NODES = "/Users/namjoolee/Downloads/networks-science-course/practicum/data/webspam_uk2007-nodes.csv.gz"
EDGES = "/Users/namjoolee/Downloads/networks-science-course/practicum/data/webspam_uk2007-edges.csv.gz"

# Read hostnames

In [13]:
id2name = dict()
id2label = dict()

In [14]:
with gzip.open(NODES, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in reader: # nodeid, hostname, label
        id2name[int(record["nodeid"])] = record["hostname"]
        id2label[int(record["nodeid"])] = record["label"]

In [21]:
unlabel = 0
spam = 0
nonspam = 0
for k, v in id2label.items():
    if v == "unlabeled":
        unlabel += 1
    elif v == "spam":
        spam += 1
    elif v == "nonspam":
        nonspam += 1
print("Unlabel:", unlabel, "Spam:", spam, "Nonspam:", nonspam)

Unlabel: 108476 Spam: 344 Nonspam: 5709


# Compute the degree of each node

In [42]:
# Initialize id2degree
id2degree = {}
N = len(id2name)
for nodeid in range(N):
    id2degree[nodeid] = 0

# Update id2degree
with gzip.open(EDGES, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in tqdm_notebook(reader): # source, destination, weight
        id2degree[int(record["source"])] += 1

# Compute pageRank

In [56]:
ITERATIONS = 20
ALPHA = 0.85

In [113]:
# Initialize pagerank
pagerank = {}
pagerank_aux = {}

for nodeid in range(N):
    pagerank[nodeid] = 1/N
    pagerank_aux[nodeid] = 0

# PageRank
for iterations in tqdm_notebook(range(ITERATIONS)):
    # PageRank
    with gzip.open(EDGES, "rt", encoding="utf-8") as input_file:
        reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
        for record in tqdm_notebook(reader): # source, destination, weight
            pagerank_aux[int(record["destination"])] += pagerank[int(record["source"])] / id2degree[int(record["source"])]

    # Set pagerank of every node to alpha x pagerank_aux + (1.0-alpha) x (1.0/N)
    for nodeid in range(N):
        pagerank[nodeid] = ALPHA * pagerank_aux[nodeid] + (1 - ALPHA) * (1 / N)

    # Set pagerank_aux to 0.0
    for nodeid in range(N):
        pagerank_aux[nodeid] = 0

# Nodes with largest values of PageRank

In [114]:
pagerank

{0: 1.327597576743937e-06,
 1: 1.7857687005288665e-06,
 2: 1.3140796305702508e-06,
 3: 1.6904131693228328e-06,
 4: 1.6901899498142543e-06,
 5: 2.2718439704617685e-06,
 6: 2.227057467610064e-06,
 7: 1.6901899498142543e-06,
 8: 2.3301667296343765e-06,
 9: 2.0197127969909436e-06,
 10: 1.6901899498142543e-06,
 11: 1.6901899498142543e-06,
 12: 1.6901899498142543e-06,
 13: 2.8354397138978697e-06,
 14: 1.3165029655767342e-06,
 15: 1.4939288736949103e-06,
 16: 1.5858268119680258e-06,
 17: 1.6664276309202398e-06,
 18: 6.7787400293115e-06,
 19: 1.6901899498142543e-06,
 20: 1.3097119506849795e-06,
 21: 1.9026512262270759e-06,
 22: 1.3201509306520773e-06,
 23: 2.3635619424736442e-06,
 24: 1.941725732491011e-06,
 25: 2.832659898412714e-06,
 26: 1.6901899498142543e-06,
 27: 1.3097119506849795e-06,
 28: 1.9386737922385977e-06,
 29: 1.3097119506849795e-06,
 30: 1.3100726356601376e-06,
 31: 1.393702473895101e-06,
 32: 4.04087185974521e-06,
 33: 6.682876286892242e-05,
 34: 1.6901899498142543e-06,
 35: 2

In [115]:
hosts_by_score = sorted(enumerate(pagerank.values()), key=lambda x: x[1], reverse=True)

In [116]:
hosts_by_score

[(81634, 0.0015168643782398274),
 (10990, 0.0014182182459715198),
 (60715, 0.0009654563807161581),
 (42757, 0.00089555331065544),
 (39774, 0.0008937889977065464),
 (5169, 0.000780473245157769),
 (40999, 0.0007209475362666682),
 (40559, 0.000697529301060852),
 (51107, 0.0006817074015284916),
 (77619, 0.0006581577785798681),
 (34406, 0.0006554311467163489),
 (19504, 0.0006482874881544456),
 (54071, 0.0006028805944996983),
 (40612, 0.0005906484974062024),
 (58631, 0.0005818417136696459),
 (59870, 0.0005757813160594587),
 (48290, 0.000540229700172051),
 (77627, 0.0005155226994647975),
 (59140, 0.000483592734339035),
 (5056, 0.00045848917724008694),
 (40577, 0.0004572991098763979),
 (38876, 0.00044551620266407174),
 (43141, 0.00043753647957066734),
 (10541, 0.0004268162067476992),
 (68475, 0.0004196267367553457),
 (82999, 0.0004187763706452809),
 (61620, 0.0003958947431418644),
 (98926, 0.00037184244017089766),
 (58593, 0.0003704741179414513),
 (78345, 0.0003640398196569995),
 (90540, 0.000

In [117]:
top20 = []
for k, v in hosts_by_score[:20]:
    top20.append(k)
print(top20)


for n in top20:
    print(id2name[n])

[81634, 10990, 60715, 42757, 39774, 5169, 40999, 40559, 51107, 77619, 34406, 19504, 54071, 40612, 58631, 59870, 48290, 77627, 59140, 5056]
www.opsi.gov.uk
www.adobe.co.uk
www.ico.gov.uk
www.dti.gov.uk
www.defra.gov.uk
news.bbc.co.uk
www.direct.gov.uk
www.dfes.gov.uk
www.fsa.gov.uk
www.nationalrail.co.uk
www.communities.gov.uk
www.bbc.co.uk
www.google.co.uk
www.dh.gov.uk
www.hmso.gov.uk
www.hse.gov.uk
www.fco.gov.uk
www.nationaltrust.org.uk
www.homeoffice.gov.uk
mysite.wanadoo-members.co.uk


# Non-spam PageRank

In [118]:
# Initialize id2nsdegree
id2nsdegree = {}
N = len(id2name)
for nodeid in range(N):
    id2nsdegree[nodeid] = 0

# Update id2nsdegree
with gzip.open(EDGES, "rt", encoding="utf-8") as input_file:
    reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
    for record in tqdm_notebook(reader): # source, destination, weight
        if id2label[int(record["source"])] != "spam" and id2label[int(record["destination"])] != "spam":
            id2nsdegree[int(record["source"])] += 1

In [119]:
ITERATIONS = 20
ALPHA = 0.85

In [120]:
# Initialize pagerank
nspagerank = {}
nspagerank_aux = {}

for nodeid in range(N):
    nspagerank[nodeid] = 1/N
    nspagerank_aux[nodeid] = 0

# nsPageRank
for iterations in tqdm_notebook(range(ITERATIONS)):
    # nsPageRank
    with gzip.open(EDGES, "rt", encoding="utf-8") as input_file:
        reader = csv.DictReader(input_file, delimiter=',', quotechar='"')
        for record in tqdm_notebook(reader): # source, destination, weight
            if id2label[int(record["source"])] != "spam" and id2label[int(record["destination"])] != "spam":
                nspagerank_aux[int(record["destination"])] += nspagerank[int(record["source"])] / id2nsdegree[int(record["source"])]

    # Set pagerank of every node to alpha x pagerank_aux + (1.0-alpha) x (1.0/N)
    for nodeid in range(N):
        nspagerank[nodeid] = ALPHA * nspagerank_aux[nodeid] + (1 - ALPHA) * (1 / N)

    # Set pagerank_aux to 0.0
    for nodeid in range(N):
        nspagerank_aux[nodeid] = 0

In [121]:
nspagerank

{0: 1.3280951118362485e-06,
 1: 1.7870464614473133e-06,
 2: 1.3141038493141733e-06,
 3: 1.6916829916852722e-06,
 4: 1.691459460792342e-06,
 5: 2.2804961715633e-06,
 6: 2.2287689447984477e-06,
 7: 1.691459460792342e-06,
 8: 2.3317257008916797e-06,
 9: 2.0209860214827573e-06,
 10: 1.691459460792342e-06,
 11: 1.691459460792342e-06,
 12: 1.691459460792342e-06,
 13: 2.830630162407429e-06,
 14: 1.315973018054369e-06,
 15: 1.493959217877803e-06,
 16: 1.5858162172184367e-06,
 17: 1.6675784571567685e-06,
 18: 6.784052121903951e-06,
 19: 1.691459460792342e-06,
 20: 1.3097119506849795e-06,
 21: 1.9051044310896894e-06,
 22: 1.3202003708426766e-06,
 23: 2.3664510370172696e-06,
 24: 1.943266302576668e-06,
 25: 2.8326608612998084e-06,
 26: 1.691459460792342e-06,
 27: 1.3097119506849795e-06,
 28: 1.9412591951931488e-06,
 29: 1.3097119506849795e-06,
 30: 1.3100715848233984e-06,
 31: 1.3946364460140623e-06,
 32: 4.043817557373268e-06,
 33: 6.693988357843838e-05,
 34: 1.691459460792342e-06,
 35: 2.353799

In [122]:
pagerank

{0: 1.327597576743937e-06,
 1: 1.7857687005288665e-06,
 2: 1.3140796305702508e-06,
 3: 1.6904131693228328e-06,
 4: 1.6901899498142543e-06,
 5: 2.2718439704617685e-06,
 6: 2.227057467610064e-06,
 7: 1.6901899498142543e-06,
 8: 2.3301667296343765e-06,
 9: 2.0197127969909436e-06,
 10: 1.6901899498142543e-06,
 11: 1.6901899498142543e-06,
 12: 1.6901899498142543e-06,
 13: 2.8354397138978697e-06,
 14: 1.3165029655767342e-06,
 15: 1.4939288736949103e-06,
 16: 1.5858268119680258e-06,
 17: 1.6664276309202398e-06,
 18: 6.7787400293115e-06,
 19: 1.6901899498142543e-06,
 20: 1.3097119506849795e-06,
 21: 1.9026512262270759e-06,
 22: 1.3201509306520773e-06,
 23: 2.3635619424736442e-06,
 24: 1.941725732491011e-06,
 25: 2.832659898412714e-06,
 26: 1.6901899498142543e-06,
 27: 1.3097119506849795e-06,
 28: 1.9386737922385977e-06,
 29: 1.3097119506849795e-06,
 30: 1.3100726356601376e-06,
 31: 1.393702473895101e-06,
 32: 4.04087185974521e-06,
 33: 6.682876286892242e-05,
 34: 1.6901899498142543e-06,
 35: 2

# Compute spam gain

In [125]:
pagerank_gain = {}
for k, v in nspagerank.items():
    pagerank_gain[k] = pagerank[k] / v

In [126]:
pagerank_gain

{0: 0.9996253769117307,
 1: 0.999284987298309,
 2: 0.9999815701446007,
 3: 0.9992493733349093,
 4: 0.9992494582297036,
 5: 0.9962060006022284,
 6: 0.9992320975252378,
 7: 0.9992494582297036,
 8: 0.9993314088116337,
 9: 0.9993699983679849,
 10: 0.9992494582297036,
 11: 0.9992494582297036,
 12: 0.9992494582297036,
 13: 1.0016991098145969,
 14: 1.000402703942326,
 15: 0.9999796887475042,
 16: 1.0000066809441561,
 17: 0.9993098818040077,
 18: 0.9992169734994665,
 19: 0.9992494582297036,
 20: 1.0,
 21: 0.998712299009661,
 22: 0.9999625509947647,
 23: 0.9987791445931343,
 24: 0.9992072264703946,
 25: 0.9999996600768176,
 26: 0.9992494582297036,
 27: 1.0,
 28: 0.9986681825070279,
 29: 1.0,
 30: 1.0000008021216178,
 31: 0.999330311407227,
 32: 0.9992715552602793,
 33: 0.9983399924891451,
 34: 0.9992494582297036,
 35: 0.9989921035790521,
 36: 0.9992502389025225,
 37: 0.9992494582297036,
 38: 0.9992494582297036,
 39: 1.1054400362576784,
 40: 0.9992479061577686,
 41: 0.9992494450309699,
 42: 0.99

In [130]:
hosts_by_score = sorted(enumerate(pagerank_gain.values()), key=lambda x: x[1], reverse=True)

In [131]:
hosts_by_score

[(46303, 33.14277368425596),
 (75287, 29.058929855290366),
 (98924, 17.898491711374145),
 (107646, 13.6422600947175),
 (95131, 10.800429380479825),
 (52597, 10.417142543549902),
 (62200, 10.353438590154301),
 (34635, 10.069001846752684),
 (11757, 9.320732629825939),
 (1917, 8.869703491243296),
 (66838, 8.452918216407495),
 (74091, 8.003510769421466),
 (104480, 7.8800664219899685),
 (72682, 7.824324020347563),
 (100812, 7.7418348004899045),
 (87838, 7.67197459115261),
 (57785, 7.531583765512691),
 (43766, 6.78796705669567),
 (64095, 6.666666666666665),
 (60824, 5.9640753129903645),
 (2176, 5.781239423370037),
 (8270, 5.247951217311628),
 (50835, 5.191675644980452),
 (6677, 4.8077477463500475),
 (41899, 4.731661403819078),
 (73010, 4.67952414423898),
 (1629, 4.6278135904554185),
 (77275, 4.43100182277229),
 (66419, 4.193015991391419),
 (35066, 4.0850428098714),
 (53609, 4.024904169484384),
 (79962, 3.9816929626743005),
 (3663, 3.853837778782957),
 (6284, 3.8396049927963523),
 (43263, 3.7

In [132]:
top20 = []
for k, v in hosts_by_score[:20]:
    top20.append(k)
print(top20)


for n in top20:
    print(id2name[n])

[46303, 75287, 98924, 107646, 95131, 52597, 62200, 34635, 11757, 1917, 66838, 74091, 104480, 72682, 100812, 87838, 57785, 43766, 64095, 60824]
www.escortnet.co.uk
www.missionfish.org.uk
www.statistics.006.free-counter.co.uk
www.uk-shoponline.co.uk
www.shop.co.uk
www.geordie-girls.co.uk
www.into.demon.co.uk
www.computerarts.co.uk
www.aili.co.uk
connect4fun.co.uk
www.kompass.co.uk
www.mercurywd.co.uk
www.theshopping-centre.co.uk
www.markwarner.co.uk
www.suppliersnearby.co.uk
www.quality-site-finder.co.uk
www.hertfordshiremobilediscos.co.uk
www.eastwoodtoday.co.uk
www.jlc.me.uk
www.ideas21.co.uk
